In [ ]:
# !pip install ipykernel

# !python -m ipykernel install --user --name practice_venv --display-name "Python (practice_venv)"

!pip install fastapi uvicorn sqlalchemy psycopg2-binary python-dotenv

# FastAPI : Used to create APIs (backend logic)
# Uvicorn : Runs your FastAPI app (like a motor starting a bike)
# SQLAlchemy : Used to talk to the database using Python instead of raw SQL
# psycopg2-binary : The connector that lets Python connect to PostgreSQL
# python-dotenv : Loads secret values (DB password, keys) from a .env file safely

In [ ]:

# database.py

from sqlalchemy import create_engine;  # Connects Python to the Database using URL
from sqlalchemy.orm import sessionmaker, declarative_base; # Creates Database Sessions, Base Table structure
from dotenv import load_dotenv; # To load env URL file
import os; 

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(bind=engine, autoflush=False)
Base = declarative_base()

In [ ]:
# table.py

from sqlalchemy import Column, Integer, String, Boolean
from database import Base

class TableStructure(Base): # just an empty table
    __tablename__ = "fast_table"

    id = Column(Integer, primary_key=True, index=True)
    title = Column(String, nullable=False)
    description = Column(String)
    status = Column(Boolean, default=False)


In [ ]:
# schema.py

from pydantic import BaseModel

class TodoBase(BaseModel):   #Common fields Used by both input and output schemas
    title : str
    description : str | None = None
    status : bool = False

class TodoCreate(TodoBase):  # To Create it uses TodoBase for Structure 
    pass                     # POST JSON → TodoCreate (Pydantic) → SQLAlchemy Model → Database

class TodoSent(TodoBase):   # TO Send request back it use TodoBase + id also its SQLAlchemy object → JSON safely so orm_mode = True
                            # Database → SQLAlchemy Object → TodoSent (orm_mode=True) → JSON Response
    id : int
    class Config:
        orm_mode = True

In [ ]:
from fastapi import FastAPI, Depends, HTTPException
from schema import TodoSent, TodoCreate
from sqlalchemy.orm import Session
from database import SessionLocal, Base, engine
from table import TableStructure

Base.metadata.create_all(bind=engine)
app = FastAPI()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# CREATE
@app.post("/fast_table", response_model=TodoSent)
def create(todo: TodoCreate, db: Session = Depends(get_db)):
    db_todo = TableStructure(**todo.model_dump())
    db.add(db_todo)
    db.commit()
    db.refresh(db_todo)
    return db_todo

# READ ALL
@app.get("/fast_table", response_model=list[TodoSent])
def get_all_rows(db: Session = Depends(get_db)):
    return db.query(TableStructure).all()

# READ ONE
@app.get("/fast_table/{id}", response_model=TodoSent)
def get_one_row(id: int, db: Session = Depends(get_db)):
    fetch = db.query(TableStructure).filter(TableStructure.id == id).first()
    if not fetch:
        raise HTTPException(status_code=404, detail="ID not found")
    return fetch

# UPDATE
@app.put("/fast_table/{id}", response_model=TodoSent)
def update_row(id: int, updated: TodoCreate, db: Session = Depends(get_db)):
    upd = db.query(TableStructure).filter(TableStructure.id == id).first()
    if not upd:
        raise HTTPException(status_code=404, detail="ID not found")

    for key, value in updated.model_dump().items():
        setattr(upd, key, value)

    db.commit()
    db.refresh(upd)
    return upd

# DELETE
@app.delete("/fast_table/{id}")
def delete_row(id: int, db: Session = Depends(get_db)):
    del_this = db.query(TableStructure).filter(TableStructure.id == id).first()
    if not del_this:
        raise HTTPException(status_code=404, detail="ID not found")

    db.delete(del_this)
    db.commit()
    return {"message": "ID deleted successfully"}


In [ ]:
curl -X POST "http://localhost:8000/fast_table" ^
 -H "Content-Type: application/json" ^
 -d "{\"title\":\"Curl User\",\"description\":\"Created from curl\",\"status\":true}"
